In [20]:
data Exp = Val Int     -- n
         | Var String
         | Add Exp Exp -- e1 + e2
         | Sub Exp Exp -- e1 - e2
         | Mul Exp Exp -- e1 * e2
         | Div Exp Exp -- e1 / e2
         deriving Show

type Env = [ (String, Int) ]

In [43]:
lookup "x" [("a",3),("b",5),("x",10),("y",22)]
lookup "z" [("a",3),("b",5),("x",10),("y",22)]

Just 10

Nothing

In [22]:
eval :: Env -> Exp -> Int
eval _   (Val n) = n
eval env (Var x) = case lookup x env of
                     Nothing -> error (x++" not defined")
                     Just v  -> v
eval env (Add e1 e2) = eval env e1 + eval env e2
eval env (Sub e1 e2) = eval env e1 - eval env e2
eval env (Mul e1 e2) = eval env e1 * eval env e2
eval env (Div e1 e2) = eval env e1 `div` eval env e2

In [23]:
eval [] (Mul (Val 5) (Add (Val 2) (Val 3))) -- 5 * (2 + 3)

25

In [24]:
eval [] (Div (Val 5) (Sub (Val 3) (Val 2))) -- 5 * (3 - 2)

5

In [25]:
eval [] (Div (Val 5) (Sub (Val 3) (Val 3))) -- 5 * (3 - 3)

: 

In [30]:
eval [] (Div (Val 5) (Var "x")) -- 5 * x

: 

In [37]:
-- Monad는 >>= 와 return 두 개의 오버로드된 메소드 제공
evm :: Env -> Exp -> Maybe Int
evm env (Val n) = return n -- Just n
evm env (Var x) = lookup x env
evm env (Add e1 e2) = (+) <$> evm env e1 <*> evm env e2
evm env (Sub e1 e2) = (-) <$> evm env e1 <*> evm env e2
evm env (Mul e1 e2) = (*) <$> evm env e1 <*> evm env e2
evm env (Div e1 e2) = do v1 <- evm env e1
                         v2 <- evm env e2
                         if v2==0 then Nothing
                                  else return (v1 `div` v2)
{-
  evm e1 >>= \v1 ->
  evm e2 >>= \v2 ->
  if v2==0 then Nothing
           else Just (v1 `div` v2)
-}
{-
  case m1 of
    Nothing -> Nothing
    Just v1 -> case m2 of
                 Nothing -> Nothing
                 Just v2 -> if v2==0
                               then Nothing
                               else Just (v1 `div` v2)
  where m1 = evm e1
        m2 = evm e2
-}

In [38]:
evm [] (Div (Val 5) (Sub (Val 3) (Val 2))) -- 5 * (3 - 2)
evm [] (Div (Val 5) (Sub (Val 3) (Val 3))) -- 5 * (3 - 3)
evm [] (Div (Val 5) (Var "x")) -- 5 * x

Just 5

Nothing

Nothing

```haskell
data Either a b = Left a | Right b
```

In [39]:
eve :: Env -> Exp -> Either String Int
eve env (Val n) = return n -- Right n
eve env (Var x) = case lookup x env of
                    Nothing -> Left (x++" not defined")
                    Just v  -> return v
eve env (Add e1 e2) = (+) <$> eve env e1 <*> eve env e2
eve env (Sub e1 e2) = (-) <$> eve env e1 <*> eve env e2
eve env (Mul e1 e2) = (*) <$> eve env e1 <*> eve env e2
eve env (Div e1 e2) = do v1 <- eve env e1
                         v2 <- eve env e2
                         if v2==0 then Left "Div by 0"
                                  else return (v1 `div` v2)

In [40]:
eve [] (Div (Val 5) (Sub (Val 3) (Val 2))) -- 5 * (3 - 2)
eve [] (Div (Val 5) (Sub (Val 3) (Val 3))) -- 5 * (3 - 3)
eve [] (Div (Val 5) (Var "x")) -- 5 * x

Right 5

Left "Div by 0"

Left "x not defined"